In [2]:
import os
import sys

In [3]:
print(f"Python version info: {sys.version}")

Python version info: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [4]:
print(os.environ.get("MPTOKEN"))

None
